In [89]:
import os
import logging 
import re


In [93]:
import logging
def clear_existing_loggers():
    """Clear all existing loggers to prevent conflicts."""
    for handler in logging.root.handlers[:]:
        logging.root.removeHandler(handler)

def setup_logger(logfile_path):
    """Setup logger to always print time and level."""
    clear_existing_loggers()
    logging.basicConfig(
        format='%(asctime)s %(levelname)-8s %(message)s',
        level=logging.INFO,
        datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[
            logging.FileHandler(logfile_path),
            logging.StreamHandler()
        ]
    )

def log_w_indent(text, indent, flag, symbol='>>'):
    """Log and add indent."""
    ind2col = {i: f"\x1b[{a}m" for i, a in enumerate([
        '1', '31', '33', '34', '35'])}
    reset = "\x1b[0m"

    if indent < 0 or indent >= len(ind2col):
        raise ValueError("Invalid indent level")

    if indent > 0:
        if flag == 'Question':
            prefix = '  Q:'
        elif flag == 'Response':
            prefix = '  LLM-A:'
        elif flag == 'Code':
            prefix = '  Code script test:'
        else:
            raise ValueError("Invalid flag")

        prefix += ind2col[indent] + (indent * 2) * symbol + ' ' + text + reset
    else:
        prefix = ind2col[indent] + text + reset

    logging.info(prefix)

setup_logger(r"D:\GXQ\human_seg\neuron_seg_human\llm_neuron_seg\utils\log_utils\log11.log")
log_w_indent("This is a test question", 1, "Question")
log_w_indent("This is a test response", 1, "Response")

2024-07-21 19:28:55 INFO       Q:>>>> This is a test question
2024-07-21 19:28:55 INFO       LLM-A:>>>> This is a test response


In [86]:
#配置openai客户端
from openai import OpenAI
# from openai.error import APIConnectionError
#from dotenv import load_dotenv
import openai
import os
import json
#config.json 进行配置用户openai账户 
with open(r'D:\GXQ\human_seg\neuron_seg_human\llm_neuron_seg\utils\llm\llm_config.json') as config_file:
    config = json.load(config_file)

CLIENT = OpenAI(base_url=config['OPENAI_API_BASE'],
                api_key=config['OPENAI_API_KEY'])

###任务描述
###分步骤描述
###工具插件描述
/*通过清晰地分步骤描述任务，并提供必要的工具插件信息，有效利用LLM完成具体任务，生成每一步的代码。*/
/*确保输入结构清晰，详细描述每个步骤的要求和工具的使用方法。使得模型的输出将更加准确和实用。*/

whole-> 3个stage 实现
FOR EACH STAGE:
  1/输入初步任务描述及可使用工具信息
      - llm返回任务基本描述及规划步骤
      - 人工修改步骤并确认
  2/每一步骤逐步向llm输入并获取response代码 
      - 代码块增量进行测试
      - 代码在10个用例上测试通过进行步骤代码留存
      - 不通过，返回报错 修改代码
  3/迭代完单stage 进行批量数据测试

In [62]:
# 分离文本+提取代码块
response = """
Here is an example of Python code:

```python
import pandas as pd

# Read CSV file
data = pd.read_csv('input.csv')
print(data.head())```
You can use the above code to read a CSV file.

Another code snippet:

python
复制代码
from data_cleaning_tool import clean_data

# Clean data
cleaned_data = clean_data(data)
print(cleaned_data.head())
Make sure to install the necessary libraries before running the code.
"""


def extract_code_blocks(response):
    # 正则表达式匹配代码块
    code_pattern = re.compile(r'```(?:\w+)?\n(.*?)```', re.DOTALL)
    code_blocks = code_pattern.findall(response)
    non_code_content = code_pattern.sub('', response)
    non_code_content = non_code_content.strip()
    
    return code_blocks, non_code_content

# 使用函数提取代码块和其他内容
code_blocks, non_code_content = extract_code_blocks(response)

print("Code Blocks:")
for i, code in enumerate(code_blocks, 1):
    print(f"Code Block {i}:\n{code}\n")

print("Non-Code Content:")
print(non_code_content)


Code Blocks:
Code Block 1:
import pandas as pd

# Read CSV file
data = pd.read_csv('input.csv')
print(data.head())

Non-Code Content:
Here is an example of Python code:


You can use the above code to read a CSV file.

Another code snippet:

python
复制代码
from data_cleaning_tool import clean_data

# Clean data
cleaned_data = clean_data(data)
print(cleaned_data.head())
Make sure to install the necessary libraries before running the code.


In [97]:
import hashlib
from collections import defaultdict
import subprocess
import json
with open(r'D:\GXQ\human_seg\neuron_seg_human\llm_neuron_seg\utils\llm\instruct_template\template.json') as templatejson_file:
    template_json = json.load(templatejson_file)

class SpoofData:
    def __getitem__(self, item):
        return f'<{item}>'
def md5hash(string):
    return int(hashlib.md5(string.encode('utf-8')).hexdigest(), 16)
#llm问答
class LLMResponse:
    def __init__(self,ini_task:str,code_save_dir):
        self.ini_task_question=ini_task
        self.ini_step_description=None
        self.step_list = None
        self.manual_list=None
        # self.n_regenerate=None
        # self.reset=reset
        #      #迭代十次不通过 记录step_id 在reset 
        # self.reset_stages=reset_stages
        #self.export_predict_response = defaultdict(defaultdict(list).copy)
        self.code_scripts=None
        self.code_save_dir = code_save_dir
        
        
        
    #indent用于日志记录 stepid:步骤id  reuse:是否进行再次输入迭代
    def llw_w_log(self, prompt, indent,stepidx, stage,reuse=False):
        #reset.stages 重置阶段列表     拿到需要重新run的stepidx
        if (stage in self.reset_stages) and (reset := self.reset.get(stepidx, {})):
            if self.accept_restore_failure and not len(reset[md5hash(prompt)]):
                logging.warning('Spoofing aborted!')
                predict_response =llm_w_log(prompt, indent)
            else:
                log_w_indent(f'Spoofed Input: {prompt}', indent,flag='Question')
                predict_response= reset[md5hash(prompt)].pop(0)
                log_w_indent(f'Spoofed Output: {predict_response}', indent, flag='Question',symbol='xx')
    
        else:
            predict_response = llm_w_log(prompt, indent)

        self.export_predict_response[stepidx][md5hash(prompt)].append(predict_response)

        return predict_response
    # 生成step列表 eachstep 进行prompt修饰后 进行llm输入，获取处理代码

    def get_ini_task_decription(self) -> str:
        log_w_indent(f'Input_Question: {self.ini_task_question}', indent=0 , symbol='<<<<<<',flag='Question')
        self.ini_task_description = chat_llm(prompt='Task:'+self.ini_task_question+template_json['get_ini_task'])
        log_w_indent(f'Initial_task_description: {self.ini_task_description}', indent=1 , flag='Question')
        
    
    def get_task_step(self) -> str:
        prompt='Task:'+self.ini_task_question+template_json['get_step_list']
        log_w_indent(f'Task_step_decom_prompt: {prompt}', indent=0 , flag='Question')
        step_list = chat_llm(prompt)
        # self.step_list ={f's{i+1}': value for i, value in enumerate(step_list)}
        # log_w_indent(f'Initial_task_step: {self.step_list}', indent=1 , flag='Response')
    
    # manual 
    def manual_modify_step(self,data) -> list:
         ###human modify step_list###
        del data
        step_prompt_list = self.step_list
        step_modify_list = []
        print("Original step_prompt_list:")
        for i, step in enumerate(step_prompt_list):
            print(f"{i+1}. {step}")
        print("\nPlease enter the modified steps. Press Enter to keep the original step.\n")

        # 循环获取用户输入
        for step in step_prompt_list:
            modified_step = input(f"Modify the step '{step}': ")
            if modified_step.strip():  # 如果输入不为空，去除前后的空格
                step_modify_list.append(modified_step)
            else:  
                step_modify_list.append(step)
        self.manual_list =step_modify_list
        log_w_indent(f'Manual_step_list: {self.manual_list}', indent=2 , flag='Response')
        #return step_modify_list

       
    def extract_code_blocks(self,self_response):
    # 正则表达式匹配代码块
        code_pattern = re.compile(r'```(?:\w+)?\n(.*?)```', re.DOTALL)
        code_blocks = code_pattern.findall(response)
        non_code_content = code_pattern.sub('', response)
        non_code_content = non_code_content.strip()
        self.code_script = code_blocks
       
    
        return code_blocks, non_code_content
    
    def get_step_code(self,step_id):
        step_list =self.manual_list
        step =str(step_list[step_id])
        step_response = chat_llm(prompt=step+template_json['get_step_code'])
        log_w_indent(f'{step_id}_code_script: {self.code_script}', indent=2 , flag='Code')
        return step_response
        # step_code,_=extract_code_blocks(self.response)
        # self.code_script=step_code


    # def get_all_prompts_for_log(self):
    #     # Spoof data to log prompting format.
    #     data = SpoofData()
    #     prompts = dict(
    #         gen_ini_task_decription=self.gen_ini_task_decription(data),
    #         task_step_list= self.gen_task_step(data),
    #         manual_modify_list = self.manual_modify_step(data),
    #         step_description=  self.extract_code_blocks(data),
    #         # 步骤额外信息也需记录
    #         # step_response =self.get_step_code(step_response)),
    #         # step_code,step_code_description =extract_code_blocks(data)
    #         )
    #     #logger记录所有
    
    #     return prompts

    def run_script(self, input_data, step_id, output_suffix):
        code = self.code_script
        script_dir = self.code_save_dir
        file_path = os.path.join(script_dir, f'{step_id}_script.py')
        output_file_path = os.path.join(script_dir, f'{step_id}_output.{output_suffix}')
        
        try:
            with open(file_path, "w") as file:
                file.write(code)
        except Exception as e:
            return f"Error writing to file: {e}"

        cmd_command = f"python {file_path} {input_data}"
        try:
            result = subprocess.run(cmd_command, shell=True, capture_output=True, text=True)
            result.check_returncode()  # 检查命令是否成功运行
            logging.info(f"Code script generated successfully!!{file_path}:{result.stdout}")
            return result.stdout
        except subprocess.CalledProcessError as e:
            logging.info(f"Error running script: {e.stderr.strip()}")
            return e.stderr.strip()
        except Exception as e:
            logging.info(f"Unexpected error: {str(e)}")
            return str(e)
        
    # def pass_metric(self):



ques = 'how to trace and restruct human neuron structure based on 3d tif images?'
code_save_dir = r"D:\GXQ\human_seg\neuron_seg_human\llm_neuron_seg\utils\llm"
llm_test = LLMResponse(ini_task=ques,code_save_dir=code_save_dir)
llm_test.get_ini_task_decription()
llm_test.get_task_step()
llm_test.manual_modify_step()


 


2024-07-21 19:44:27 INFO     Input_Question: how to trace and restruct human neuron structure based on 3d tif images?
2024-07-21 19:44:37 INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-21 19:44:37 INFO       Q:>>>> Initial_task_description: ### 1. Task Objectives and Function Description

- **Objectives:**
  - Reconstruct the 3D morphology of human neurons from volumetric TIFF images.
  - Generate accurate neuron tracings that can be used for further analysis, such as studying neuronal connectivity, morphology, and potential pathological changes.
  
- **Function and Purpose:**
  - Provide a computational method for researchers to transform raw 3D TIFF images into a usable format for neurobiological analysis.
  - Aid in the analysis of neuronal structures by creating 3D models that can be measured and manipulated.
  - Enhance research capabilities in neuroscience by automating a complex and time-consuming task, allowing for high-throughp

TypeError: manual_modify_step() missing 1 required positional argument: 'data'

In [87]:
import openai

# 配置api日志记录
logging.basicConfig(level=logging.WARNING, filename='chat_llm.log', filemode='a',
                    format='%(asctime)s - %(levelname)s - %(message)s')
from tenacity import (retry, wait_random_exponential)
@retry(wait=wait_random_exponential(min=1, max=10))


def chat_llm(prompt):
    """Predict with GPT-4 model."""

    if isinstance(prompt, str):
        messages = [
            {"role": "user", "content": prompt},
        ]
    else:
        messages = prompt

    try:
        output = CLIENT.chat.completions.create(
            model='gpt-4o',
            messages=messages,
        )
        response = output.choices[0].message.content
        return response
    except openai.error.APIConnectionError as e:
        # 记录日志警告及错误信息
        logging.warning(f"APIConnectionError occurred: {e}")
        return "An external error occurred while trying to connect to the API. Please try again later."


def main(prompt):
    """ log inputs and outputs."""
    log_w_indent(f'Input: {prompt}', indent=1 , flag='Question')
    response = chat_llm(prompt)
    ##处理response
    log_w_indent(f'Output: {response}', indent=2, symbol='xx',flag='Response')
    return response
main('how to trace and restruct human neuron structure based on 3d tif images?')

2024-07-21 18:42:03 INFO       Q:>>>> Input: how to trace and restruct human neuron structure based on 3d tif images?
2024-07-21 18:42:18 INFO     HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
2024-07-21 18:42:18 INFO       LLM-A:xxxxxxxx Output: Tracing and reconstructing the human neuron structure from 3D TIFF images is a sophisticated task often used in neuroscience research. This process involves several steps, including image preprocessing, neuron tracing, 3D reconstruction, and analysis. Here’s a high-level guide on how to approach this:

### 1. Prerequisites:
- **Software**: You may need specialized software such as Fiji (ImageJ), Vaa3D, NeuTube, NeuronStudio, or Aivia.
- **Computational Resources**: Depending on the size and complexity of your 3D TIFF images, substantial computational resources may be required.
- **Plugins and Scripts**: Sometimes, you might need specific plugins or custom scripts to handle the data.

### 2. Image Preprocessing

'Tracing and reconstructing the human neuron structure from 3D TIFF images is a sophisticated task often used in neuroscience research. This process involves several steps, including image preprocessing, neuron tracing, 3D reconstruction, and analysis. Here’s a high-level guide on how to approach this:\n\n### 1. Prerequisites:\n- **Software**: You may need specialized software such as Fiji (ImageJ), Vaa3D, NeuTube, NeuronStudio, or Aivia.\n- **Computational Resources**: Depending on the size and complexity of your 3D TIFF images, substantial computational resources may be required.\n- **Plugins and Scripts**: Sometimes, you might need specific plugins or custom scripts to handle the data.\n\n### 2. Image Preprocessing:\nBefore tracing the neurons, you need to preprocess the images to enhance the quality and make the structures more distinguishable.\n\n- **Noise Reduction**: Apply filters like Gaussian blur or median filter to reduce noise.\n- **Contrast Enhancement**: Use histogram equ